# Importing Lines

In [1]:
import os, sys

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split, KFold

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('../utils/'))))
from utils.saving import *
from utils.load_data import *

# Load Data

In [2]:
CTYPE = 'multi'

X = loader.image()
y = loader.label(return_type=CTYPE)

In [3]:
SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [4]:
trn_ds = TensorDataset(X_train, y_train)
tst_ds = TensorDataset(X_test, y_test)

BATCH = 16
trn_dl = DataLoader(trn_ds, batch_size=BATCH, shuffle=True)
tst_dl = DataLoader(tst_ds, batch_size=BATCH, shuffle=True)

# Build Model

In [5]:
from convnet_v2 import *

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [8]:
CTYPE = 'multi'
# model = ConvNet(ctype=CTYPE, flat_nodes=19200).to(device)
model = ConvNet(ctype=CTYPE).to(device)

In [9]:
weight = loader.weighting(y).to(device)

In [10]:
loss_fn = model.loss_fn(weight=weight)

LEARNING_RATE = 1e-2
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [11]:
accuracy = model.accuracy

## Debug

In [12]:
model

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(6, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1280, out_features=

In [14]:
for idx, (x, y) in enumerate(trn_dl):
    if idx < 3:
        x = x.to(device)
        print(model(x))

G:\My Drive\brain_data\workspace\Convolution_Models\convnet_v2.py:80: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.last_layer(out)


tensor([[0.0651, 0.1147, 0.1349, 0.0750, 0.2063, 0.0698, 0.0677, 0.1174, 0.0847,
         0.0644],
        [0.0734, 0.1909, 0.1576, 0.0918, 0.1430, 0.0652, 0.0484, 0.0993, 0.0776,
         0.0529],
        [0.0805, 0.1304, 0.1414, 0.1135, 0.1424, 0.0875, 0.0601, 0.0856, 0.0810,
         0.0775],
        [0.1144, 0.1380, 0.1605, 0.0803, 0.1624, 0.1017, 0.0329, 0.0860, 0.0617,
         0.0622],
        [0.0746, 0.1752, 0.1343, 0.0969, 0.1592, 0.0627, 0.0645, 0.1146, 0.0702,
         0.0477],
        [0.0761, 0.1438, 0.1104, 0.1135, 0.1494, 0.0750, 0.0600, 0.0869, 0.0875,
         0.0974],
        [0.0909, 0.1858, 0.1560, 0.0721, 0.1010, 0.1146, 0.0771, 0.0827, 0.0605,
         0.0592],
        [0.0886, 0.1368, 0.1048, 0.0879, 0.1823, 0.1127, 0.0906, 0.0631, 0.0497,
         0.0836],
        [0.0640, 0.1532, 0.1597, 0.1094, 0.1571, 0.0777, 0.0559, 0.0973, 0.0826,
         0.0431],
        [0.0640, 0.1105, 0.1334, 0.1125, 0.2428, 0.0979, 0.0650, 0.0860, 0.0525,
         0.0352],
        [0

In [15]:
total_params = sum(p.numel() for p in model.parameters())
print("Num of Total Parameter : ",total_params)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Num of Trainable Parameter :",trainable_params)

Num of Total Parameter :  153350
Num of Trainable Parameter : 153350


## Train

### Holdout

In [16]:
EPOCHS = range(200)

verbose = 3

trn_losses, val_losses = [], []
trn_accs, val_accs = [], []

for epoch in EPOCHS:

    trn_bth_loss, trn_bth_acc = 0, 0
    trn_cnt = 0
    model.train()
    for x, y in trn_dl:
        
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        y_pred = model(x).to(device)

        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()

        trn_bth_loss += loss.item()
        acc, cnt = accuracy(y_pred, y)
        trn_bth_acc += acc
        trn_cnt += cnt

    trn_losses.append(trn_bth_loss)
    trn_accs.append(trn_bth_acc / trn_cnt * 100)


    val_bth_loss, val_bth_acc = 0, 0
    val_cnt = 0
    model.eval()
    with torch.no_grad():
        for x, y in tst_dl:
            
            x, y = x.to(device), y.to(device)

            y_pred = model(x).to(device)

            loss = loss_fn(y_pred, y)

            val_bth_loss += loss.item()
            acc, cnt = accuracy(y_pred, y)
            val_bth_acc += acc
            val_cnt += cnt

        val_losses.append(val_bth_loss)
        val_accs.append(val_bth_acc / val_cnt * 100)

    if verbose == 0:
        if epoch % 5 == 0:
            print(f'EPOCHS {epoch} :: [LOSS] {val_losses[-1]:.3f} | [ACC] {val_accs[-1]:.3f}%')

    elif verbose == 1:
        print(f'EPOCHS {epoch} :: [LOSS] {val_losses[-1]:.3f} | [ACC] {val_accs[-1]:.3f}%')

    elif verbose == 2:
        print(f'EPOCHS {epoch}')
        print(f'TRAIN :: [LOSS] {trn_losses[-1]:.3f} | [ACC] {trn_accs[-1]:.3f}%')
        print(f'VALID :: [LOSS] {val_losses[-1]:.3f} | [ACC] {val_accs[-1]:.3f}%\n')

    elif verbose == 3:
        print(f'EPOCHS {epoch}')
        print(f'TRAIN :: [LOSS] {trn_losses[-1]:.3f} | [ACC] {trn_accs[-1]:.3f}%, {trn_bth_acc}/{trn_cnt}')
        print(f'VALID :: [LOSS] {val_losses[-1]:.3f} | [ACC] {val_accs[-1]:.3f}%, {val_bth_acc}/{val_cnt}\n')

EPOCHS 0
TRAIN :: [LOSS] 103.810 | [ACC] 9.812%, 68/693
VALID :: [LOSS] 26.104 | [ACC] 9.195%, 16/174

EPOCHS 1
TRAIN :: [LOSS] 103.828 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.087 | [ACC] 9.195%, 16/174

EPOCHS 2
TRAIN :: [LOSS] 103.845 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.087 | [ACC] 9.195%, 16/174

EPOCHS 3
TRAIN :: [LOSS] 103.859 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.091 | [ACC] 9.195%, 16/174

EPOCHS 4
TRAIN :: [LOSS] 103.852 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.099 | [ACC] 9.195%, 16/174

EPOCHS 5
TRAIN :: [LOSS] 103.737 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.074 | [ACC] 9.195%, 16/174

EPOCHS 6
TRAIN :: [LOSS] 103.855 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.077 | [ACC] 9.195%, 16/174

EPOCHS 7
TRAIN :: [LOSS] 103.814 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.079 | [ACC] 9.195%, 16/174

EPOCHS 8
TRAIN :: [LOSS] 103.845 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.090 | [ACC] 9.195%, 16/174

EPOCHS 9
TRAIN :: [LOSS] 103.850 | [ACC] 10.390%, 72/693
VALID ::

EPOCHS 79
TRAIN :: [LOSS] 103.682 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.074 | [ACC] 9.195%, 16/174

EPOCHS 80
TRAIN :: [LOSS] 103.561 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.086 | [ACC] 9.195%, 16/174

EPOCHS 81
TRAIN :: [LOSS] 103.704 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.076 | [ACC] 9.195%, 16/174

EPOCHS 82
TRAIN :: [LOSS] 103.841 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.086 | [ACC] 9.195%, 16/174

EPOCHS 83
TRAIN :: [LOSS] 103.852 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.071 | [ACC] 9.195%, 16/174

EPOCHS 84
TRAIN :: [LOSS] 103.837 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.090 | [ACC] 9.195%, 16/174

EPOCHS 85
TRAIN :: [LOSS] 103.848 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.091 | [ACC] 9.195%, 16/174

EPOCHS 86
TRAIN :: [LOSS] 103.703 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.079 | [ACC] 9.195%, 16/174

EPOCHS 87
TRAIN :: [LOSS] 103.852 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.084 | [ACC] 9.195%, 16/174

EPOCHS 88
TRAIN :: [LOSS] 103.857 | [ACC] 10.390%, 72/6

EPOCHS 157
TRAIN :: [LOSS] 103.845 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.057 | [ACC] 9.195%, 16/174

EPOCHS 158
TRAIN :: [LOSS] 103.853 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.086 | [ACC] 9.195%, 16/174

EPOCHS 159
TRAIN :: [LOSS] 103.701 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.077 | [ACC] 9.195%, 16/174

EPOCHS 160
TRAIN :: [LOSS] 103.818 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.087 | [ACC] 9.195%, 16/174

EPOCHS 161
TRAIN :: [LOSS] 103.816 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.094 | [ACC] 9.195%, 16/174

EPOCHS 162
TRAIN :: [LOSS] 103.861 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.060 | [ACC] 9.195%, 16/174

EPOCHS 163
TRAIN :: [LOSS] 103.833 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.086 | [ACC] 9.195%, 16/174

EPOCHS 164
TRAIN :: [LOSS] 103.850 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.062 | [ACC] 9.195%, 16/174

EPOCHS 165
TRAIN :: [LOSS] 103.711 | [ACC] 10.390%, 72/693
VALID :: [LOSS] 26.074 | [ACC] 9.195%, 16/174

EPOCHS 166
TRAIN :: [LOSS] 103.694 | [ACC] 10.

In [22]:
comments = f'BATCH: {BATCH} | LR: {LEARNING_RATE} | EPOCHS: {EPOCHS}'
save_model(model, EPOCHS, optimizer, loss_fn, [trn_losses, val_losses, trn_accs, val_accs], comments)

### K-Folds

In [ ]:
EPOCHS = range(200)
FOLDS = 10

SPLITS = 10

cv_loss_trn, cv_loss_val = [], []
cv_accs_trn, cv_accs_val = [], []

verbose = 3

kfold = KFold(n_splits=SPLITS)
for i, (trn_idx, val_idx) in enumerate(kfold.split(X_train)):
    print(f'FOLD :: {i}')
    
    trn_fdl = DataLoader(TensorDataset(X_train[trn_idx], y_train[trn_idx]))
    val_fdl = DataLoader(TensorDataset(X_train[val_idx], y_train[val_idx]))
    
    trn_losses, val_losses = [], []
    trn_accs, val_accs = [], []
    
    for epoch in EPOCHS:
        
        trn_bth_loss, trn_bth_acc = 0, 0
        trn_cnt = 0
        model.train()
        for x, y in trn_fdl:
            
            optimizer.zero_grad()
            
            y_pred = model(x)
            
            loss = loss_fn(y_pred, y)
            loss.backward()
            optimizer.step()
            
            trn_bth_loss += loss.item()
            acc, cnt = accuracy(y_pred, y)
            trn_bth_acc += acc
            trn_cnt += cnt
            
        #trn_cnt = trn_fdl.__len__()
        trn_losses.append(trn_bth_loss / trn_cnt)
        trn_accs.append(trn_bth_acc / trn_cnt * 100)
        
        
        val_bth_loss, val_bth_acc = 0, 0
        val_cnt = 0
        model.eval()
        with torch.no_grad():
            for x, y in val_fdl:

                y_pred = model(x)

                loss = loss_fn(y_pred, y)

                val_bth_loss += loss.item()
                acc, cnt = accuracy(y_pred, y)
                val_bth_acc += acc
                val_cnt += cnt

            #val_cnt = val_fdl.__len__()
            val_losses.append(val_bth_loss / val_cnt)
            val_accs.append(val_bth_acc / val_cnt * 100)
        
        if verbose == 0:
            if epoch % 5 == 0:
                print(f'EPOCHS {epoch} :: [LOSS] {val_losses[-1]:.3f} | [ACC] {val_accs[-1]:.3f}%')
                
        elif verbose == 1:
            print(f'EPOCHS {epoch} :: [LOSS] {val_losses[-1]:.3f} | [ACC] {val_accs[-1]:.3f}%')
            
        elif verbose == 2:
            print(f'EPOCHS {epoch}')
            print(f'TRAIN :: [LOSS] {trn_losses[-1]:.3f} | [ACC] {trn_accs[-1]:.3f}%')
            print(f'VALID :: [LOSS] {val_losses[-1]:.3f} | [ACC] {val_accs[-1]:.3f}%\n')
            
        elif verbose == 3:
            print(f'EPOCHS {epoch}')
            print(f'TRAIN :: [LOSS] {trn_losses[-1]:.3f} | [ACC] {trn_accs[-1]:.3f}%, {trn_bth_acc}/{trn_cnt}')
            print(f'VALID :: [LOSS] {val_losses[-1]:.3f} | [ACC] {val_accs[-1]:.3f}%, {val_bth_acc}/{val_cnt}\n')
            
    cv_loss_trn.append(trn_losses)
    cv_accs_trn.append(trn_accs)
    
    cv_loss_val.append(val_losses)
    cv_accs_val.append(val_accs)

FOLD :: 0


In [ ]:
from utils.plotting_utils import *

cv_plotter(cv_loss_trn, cv_loss_val, title='loss')

In [ ]:
cv_plotter(cv_accs_trn, cv_accs_val, title='acc')

## With Hold-out Testset

In [ ]:
with torch.no_grad():
    
    tst_loss, tst_acc = 0, 0
    tst_cnt = 0
    model.eval()
    for x, y in tst_dl:
        
        y_pred = model(x)
        
        loss = loss_fn(y_pred, y)
        
        tst_loss += loss.item()
        acc, cnt = accuracy(y_pred, y)
        tst_acc += acc
        tst_cnt += cnt
    
    #tst_acc /= tst_cnt
print(f'HOLDOUT TESTSET :: [LOSS] {tst_loss:.3f} | [ACC] {tst_acc/tst_cnt * 100:.3f}% {tst_acc}/{tst_cnt}')

In [ ]:
y_pred.argmax(axis=1), y

## Save Model

In [44]:
import time
t = time.gmtime(time.time())

SAVE_PATH = f'./models/{t.tm_year}_{t.tm_mon}_{t.tm_mday}-{t.tm_hour}_{t.tm_min}.pt'
torch.save({
            'epoch': EPOCHS,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss_fn,
            }, SAVE_PATH)

In [ ]:
loading = ConvNet(CTYPE)

In [ ]:
ckpnt = torch.load(SAVE_PATH)

In [ ]:
model.load_state_dict(ckpnt['model_state_dict'])

In [ ]:
model